# Initialize

To install prerequisites, open a terminal in Jupyter and run the following commands:
```
git clone https://github.com/claudiofahey/p3_test_driver
cd p3_test_driver
pip install -e p3_data
pip install openpyxl
```
Then restart the Python kernel.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
importlib.reload(p3_data)
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*rapids*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df['compression'] = clean_df['compression'].fillna('None')
clean_df['warmup'] = clean_df['warmup'].fillna(False)
clean_df = clean_df.sort_values(['utc_begin'])

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'size_multiplier',
 'partitions',
 'stripe_size_MiB',
 'compression',
 'file_format',
 'storage_type',
 'cached',
 'warmup',
 'test',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',    
    'persist_sec',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
#clean_df[cols].to_csv('results.csv')

In [ ]:
filt1_df = filter_dataframe(
    clean_df,
#     cached=False,
    size_multiplier=3.0,
    stripe_size_MiB=(64,10240),
    warmup=False,
)
len(filt1_df)

In [ ]:
groupby_cols = [
 'size_multiplier',
 'partitions',
 'stripe_size_MiB',
 'compression',
 'file_format',
 'cached',
 'warmup',
 'storage_type',
]
df = filt1_df.fillna(0).groupby(groupby_cols).agg({'persist_sec': ['min','mean','max','size']})
df

In [ ]:
df.unstack(['storage_type'])